# Pandas Programming Tips
Developed by: Yongkang Liu (yongkang.liu.phd@gmail.com)  
Created on October 3, 2019.  
Updated on October 3, 2019.

Pandas is the de facto Python library used for storing, processing, and visualizing data in analytical works. Meanwhile, it is oftern used to shape data and feed them to machine learning and artificial intelligence algorithms. 

This notebook collects a few useful and verified methods to efficiently utilize Pandas data structures and functions. It serves as my all-in-one stop for Pandas's syntax reference, coding tips, and templates. They are based on my personal coding experience which may only work with specific coding environments or Python versions.

To look for general Python programming information, jump back to the main notebook below.  
[Python Coding Tips](Python_Coding_Tips.ipynb)

<a name="toc"></a>
# Table of contents
    
1. [Data Structure Basics](#data)
    1. [DataFrame/Array Dimensions (axis = 0, 1, ..)](#data.dimension)

1. [Pandas DataFrames](#dataframe)
    1. [Construct a DataFrame](#df.construct)
    1. [Check DataFrame in the First Place](#df.check)
    1. [Visit DataFrame Elements](#df.visit)
    1. [Update DataFrame](#df.update)
        1. [Change Column Data Type]

1. [Useful Links for Python Libraries and Toolboxes](#lib)
    1. [General Python Tips](Python_Coding_Tips.ipynb)
    1. [Markdown Tips](#lib.markdown)
    1. [Graph Database](#lib.graph_db)

1. [End: To add a new section](#end)


<a name="data"></a>
## Data Structure Basics

Refer to the following reference if there is any question about Pandas' Series and DataFrame. 

*Reference:* 
* [Intro to Data Structure in Pandas](https://pandas.pydata.org/pandas-docs/stable/getting_started/dsintro.html)
* [Modern Pandas (6 Parts)](https://tomaugspurger.github.io/modern-1-intro.html)

[Back](#toc)
<a name="data.dimension"></a>
### DataFrame/Array Dimensions (axis = 0, 1, ..)

I think this is an essential concept which should be clarified and clearly understood at the beginning of all other discussions on multi-dimentional data processing.

**df.func_foo(axis=n)** means applying the function, *func_foo*, **in the n-th dimention of array**. 

E.g., for a 3-D array with dimensions as M x N x L. The result of a function applied in the first dimension (axis=0) will be a 2-D arrary (N x L); in the second dimension (axis=1) will be a 2-D array (M x L); and in the third dimension (axis=2) will be a 2-D array (M x N). Similarly, a function applied in both (axis=0) and (axis=1) dimensions will return a 1-D array of L elements. 


For a 2-D example as follows

```python
df = pd.DataFrame(
    {
        "col_1": [1, 2, 3, 4],
        "col_2": [5, 6, 7, 8],
        "col_3": [9, 10, 11, 12]
    }
)
```
The result would be:

 |\_\_index\_\_ | col_1  | col_2  | col_3  |
 |:--- |:------:|:------:|:------:|
 |0 | 1      | 5      | 9|
 |1 |2|6|10|
 |2 |3|7|11|
 |3 |4|8|12|
 
df.shape returns 4x3, i.e., there are 4 positions/indices in the first dimension (axis=0) and 3 positions/indices in the second dimension (axis=1).
 
In a 2-D dataframe, the first dimension (axis=0) denotes rows while the second one (axis=1) refers to columns.

To determine what axis=0 or axis=1 refers to, we can follow the idea of ".iloc\[axis0\]\[axis1\]". 

For example, df.mean(axis=0) means we want to have the mean along axis=0, i.e., we will enumerate all possible "i" in the first dimension given a fixed position in the other dimension(s), i.e., **df.iloc\[i\]\[axis1\]**, and calculate the average. In the above example, each element of df.mean(axis=0) refers to the mean of a collection of data, i.e.,  

            df.iloc[0][axis1], df.iloc[1][axis1], df.iloc[2][axis1], df.iloc[3][axis1].

As the mean is calculated given each position in the second demention (axis=1), there would be in total 3 mean values, one for each column (each index in the second dimension). 

Following the same rule, df.mean(axis=1) is to calculate the mean over different positions in the dimension axis=1. Use the same example, we apply the function to

            df[axis0][0], df[axis0][1], df[axis0][2] 

given a specific/fixed index in the dimension axis=0. Since there are four indices in axis=0, the result is a vector of 4 numbers, one for each row. 

This can also apply to the other statistical functions, such as df.sum(axis=0), df.min(axis=0), where we just change the function/goal of the calculation while keeping the set/collection defined above.

Similarly, df.dropna(axis=0) contains steps:
* 1) obtain a slice of dataframe/array df\[x\]\[:\]...\[:\] for a possible value x in the dimension axis=0;
* 2) search "NaN" in the slice;
* 3) if found a "NaN", delete the whole slice;
* 4) change the value of x, and go to Step 1 until the end.

For df.dropna(axis=1), just change the slice in each search to df\[:\]\[y\]...\[:\].


[Back](#toc)
<a name="dataframe"></a>
## Pandas DataFrame

One of the basic data structure in Pandas is DataFrame.

[Back](#toc)
<a name="df.construct"></a>
### Construct a DataFrame

#### Manual Input
```python
import pandas as pd

df = pd.DataFrame(
    {
        "column_1": [1, 2, 3, 4, 5],
        "another_column": ['this', 'column', 'has', 'strings', 'inside!'],
        'float_column': [0.1, 0.2, 0.3, 33, 52.118],
        'binary_solo': [True, False, False, True, False]        
    }
)
```

Or we can construct a DataFrame from Numpy's ndarray.
```python
import pandas as pd
import numpy as np
df = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]), columns=['a', 'b', 'c'])
```

#### Import from File
For a large data set, the DataFrame is created by importing data from CSV files.
```python
import pandas as pd
df = pd.read_csv('foo_path_file.csv')
```


[Back](#toc)
<a name="df.check"></a>
### Check DataFrame in the First Place

```python
# suppose that df_foo is a DataFrame

# A overall picture of the DataFrame
df_foo.info()   

# check how many rows and columns in a DataFrame
df_foo.shape    # returns a tuple containing each dimension size. E.g., For a 2-D dataframe, it is (rows x columns)
# Each dimension can be accessed as the tuple elements. E.g., #rows=df_foo.shape[0], #cols=df_foo.shape[1]

# display the first 5 rows of data
df_foo.head()   # if the number of rows is specified, e.g., df_foo.head(n), show the top n rows

# display the last 5 rows of data
df_foo.tail()   # if the number of rows is specified, e.g., df_foo.tail(n), show the bottom n rows 

# display the data type in each column
df_foo.dtypes   # returns a Pandas Series.
# Each column type can be accessed through the call of column name. E.g., df_foo.dtypes['col_1']: dtype('int64')

# display the column names
df_foo.columns  # returns a Pandas Index
# Each column name can be obtained by the index. E.g., df_foo.columns[1]='col_1'

df_foo.describe()  # for numeric columns, returns basic statistics; for string columns, returns the value count. The return format is another DataFrame. Can also be applied to selected columns if specified, e.g., df_foo['column_1_name'].describe()

```

[Back](#toc)  
<a name="df.visit"></a>  
### Visit DataFrame Elements

We have handful options to access DataFrame elements.

```python
# Suppose we have a DataFrame object, df_sample, where each row contains multipl columns. One column name is "time"
row_index = 0
col_index = 0

df_sample.iloc[row_index][col_index] # using both row and column indices   
df_sample.loc[row_index]['time']     # using the row index and column name
de_sample.iat[row_index, col_index]  # using both row and column indices
df_sample.at[row_index, 'time']      # using the row index and column name
```

The difference between .loc\[.\] and .iloc\[.\] is:
* .loc\[.\] works on **labels** of your index. E.g., loc\[2\] -> look for the values in a DataFrame that have an index labeled 2
* .iloc\[.\] works on the **position** in your index. E.g., iloc\[2\] -> look for the values in a DataFrame that are at index 2

Options of selecting row/column in a DataFrame include:
```python
df_sample.iloc[0]             # select row by index
df_sample.loc[:, 'time']      # select column by column name
```

Another way to access a column in the DataFrame is by the column name.

```python
df_sample['time']     # returns a Series object, i.e., a column of the DataFrame
df_sample['time'][0]  # accessing a value by index

# if the column name does not contain white space(s), it can also be written in such a format
df_sample.time  
df_sample.time[0]  
```

Reference: 
1. [Pandas Tutorial: DataFrames in Python](https://www.datacamp.com/community/tutorials/pandas-tutorial-dataframe-python)
2. [Python Pandas : How to add rows in a DataFrame using dataframe.append() \& loc[], iloc[]](https://thispointer.com/python-pandas-how-to-add-rows-in-a-dataframe-using-dataframe-append-loc-iloc/)   
  

[Back](#toc)
<a name="df.update"></a>
### Update DataFrame

*reference: [The Pandas DataFrame – loading, editing, and viewing data in Python](https://www.shanelynn.ie/using-pandas-dataframe-creating-editing-viewing-data-in-python/)*

<a name="df.update.type"></a>
#### Change Column Data Type

```python
df.col_1 = df.col_1.astype('int64')
df = df.astype({"col_1":'category', "col_4":'int64'})
```


<a name="df.update.rename"></a>
#### Rename Column(s)

Use **rename()** to rename columns.

```python
df = df.rename(columns={"col_4":"fruit"})
df.rename(columns={"col_4":"fruit"}, inplace=True)
df.rename(columns={"col_3":"type", "col_4":"fruit"}, inplace=True)

# To standardize the name format, it can be useful to remove spaces and lowercase all column names
df.rename(columns=lambda x: x.lower().replace(' ', '_'))
```


<a name="df.update.delCol"></a>
#### Delete Columns in DataFrame   

To delete a column, or columns, use the name of column(s), and specify the "axis" as 1.
  
The Pandas function is **drop()**. It will return a modified DataFrame. If set "inplace=True", it will directly modify the input DataFrame.
  
```python
df = df.drop("Age", axis=1)           # remove a column with the column name "Age", and returns the result

df.drop("Age", axis=1, inplace=True)  # remove the column in the DataFrame, no returns.

df.drop(["Age", "Sex"], axis=1, inplace=True)  # remove columns
``` 

<a name="df.update.delRow"></a>
#### Delete Rows in DataFrame   

To delete rows, set the "axis" as 0 in drop().  

Removing rows is based on "labels", rather than numeric indices. To delete rows based on their numeric positions/indices, use .iloc() to reassign the DataFrame values.

```python 
df = df.drop([0,1], axis=0)            # remove the rows with labels 0 and 1 and return the updated dataframe 

df.drop([0, 1], axis=0, inplace=True)  # set inplace=True to direct apply changes to the original dataframe
```
Here is an example with extensive details.  

```python
import pandas as pd
df = pd.DataFrame(
    {
        'col_1':[1, 2, 3],
        'col_2':[2, 3, 4],
        'col_3':[4, 5, 6],
        'col_4':['apple', 'pear', 'banana']
    }
)

# To delete rows by labels in a column, SET the index first
df.set_index("col_4") # using the labels in the column "col_4"
df.drop("apple", axis=0, inplace=True) # remove all rows with label "apple" in the column "col_4"

# Delete the first four rows using iloc selector
df = df.iloc[4:,]

df = df.dropna()  # delete all rows with 'nan' values
```

After inserting or deleting rows in a DataFrame, each row still uses its original index/labels. These labels may not be continuous. For example, if the row of label '0' is deleted, we can not access the new first row by .iloc[0]. To refresh the indices, it is necessary to reset the index of the DataFrame.

```python
# suppose a DataFrame, df_foo
df_foo.drop([0,1], axis=0, inplace=True)
df_foo.reset_index(inplace=True)     # This reset the index, however, the old index will be saved in a new column 'index'
df_foo.reset_index(inplace=True, drop=True)  # drop=True can remove the former index column.
```

We can also delete rows by statements.

```python
df = df[df.col_1 != 1]  # suppose df has a column "col_1", remove all rows with the value of 1
df = df[df["col_1"] != 1]  # It also works. In case there is space in the column name, use this format
df = df[df.col_1 != '']    # if there is a NaN in the location 
```


<a name="lib"></a>
## Useful Links for Python Libraries and Toolboxes

[Back](#toc)

The following topics are expected to keep expanding and likely to spin off in separate notebooks with detailed discussions.


<a name="lib.general"></a>
### General Tips

* [Python 2 vs 3](https://www.digitalocean.com/community/tutorials/python-2-vs-python-3-practical-considerations-2)
* [Python Tips](https://book.pythontips.com/en/latest/index.html)
* [Python for Engineers](https://www.pythonforengineers.com/)

[Back](#toc)
<a name="lib.markdown"></a>
### Markdown Tips

*Reference:[Markdown Cheatsheet](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#links), [another cheatsheet](https://github.com/adam-p/markdown-here/wiki/Markdown-Here-Cheatsheet)*

#### Show Asterisk &ast; in a Markdown Cell

We can use ``` &ast; ``` in Markdown to show it.

[Back](#toc)
<a name="lib.graph_db"></a>
### Graph Database

* [Py2neo: neo4j's Python API](https://py2neo.org/v4/), [(Quick Start)](https://medium.com/neo4j/py2neo-v4-2bedc8afef2)



[Back](#toc)
<a name="lib.plot"></a>
### Plot Figures in Python

*Reference* 
* [5 Quick and Easy Data Visualizations in Python with Code](https://towardsdatascience.com/5-quick-and-easy-data-visualizations-in-python-with-code-a2284bae952f) a very interesting guide map regarding the visualization purposes

* [Matplotlib gallery](https://matplotlib.org/gallery/index.html)

<a name="end"></a>
## End of Notebook
[Back](#toc)